# ***SEM VII - COLLABORATIVE PROJECT***
### **Team Members:**
*   Sakshi Savardekar - 22102A2002
*   Anuj Ramane - 22102A2003

---




# **Project Title :** Smart News Summarizer

---



# **Installing required packages**

In [1]:
# !pip install PyPDF2
!pip install newspaper3k
!pip uninstall -y nltk
!pip install -U nltk

Found existing installation: nltk 3.9.1
Uninstalling nltk-3.9.1:
  Successfully uninstalled nltk-3.9.1
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)


# **Trying to extract text from Newpaper PDF (Have Issues, so don't use it)**

In [2]:
# import PyPDF2

# def extract_text_from_pdf(pdf_path):
#     text = ""
#     with open(pdf_path, 'rb') as file:
#         reader = PyPDF2.PdfReader(file)
#         for page_num in range(len(reader.pages)):
#             page = reader.pages[page_num]
#             text += page.extract_text()
#     return text

# # Example usage:
# pdf_path = '/content/newspaper1.pdf'
# raw_text = extract_text_from_pdf(pdf_path)
# print(raw_text[:1000])  # Print the first 500 characters to verify


# **Using NewsData API to get the news articles**

In [3]:
import requests
import json
# Function to get live stock data for a symbol
def get_news():
    api_url = "https://newsdata.io/api/1/latest?apikey=pub_541834e076195e37b19a864efef9573eefdec&language=en&country=in"
    response = requests.get(api_url)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return response.status_code

news = get_news()


In [4]:
news

{'status': 'success',
 'totalResults': 26898,
 'results': [{'article_id': 'a59e7b1da61e88616ff3fec49a0ae5d6',
   'title': "Delhi doctors demand 'justice for Abhaya', safety measures for healthcare workers",
   'link': 'https://www.deccanherald.com/india/delhi/delhi-doctors-demand-justice-for-abhaya-safety-measures-for-healthcare-workers-3237339',
   'keywords': None,
   'creator': ['PTI'],
   'video_url': None,
   'description': None,
   'content': 'ONLY AVAILABLE IN PAID PLANS',
   'pubDate': '2024-10-17 19:37:59',
   'pubDateTZ': 'UTC',
   'image_url': 'https://images.deccanherald.com/deccanherald%2F2024-10-17%2F1z3j2p0j%2FPTI03_05_2024_000124B.jpg?ar=40%3A21&auto=format%2Ccompress&mode=crop&ogImage=true&rect=0%2C0%2C4213%2C2212&w=1200',
   'source_id': 'deccanherald',
   'source_priority': 10080,
   'source_name': 'Deccan Herald',
   'source_url': 'https://www.deccanherald.com',
   'source_icon': None,
   'language': 'english',
   'country': ['india'],
   'category': ['top'],
   'ai

In [5]:
import json
import csv

data_file = open('sample_data/news_data.csv', 'w', newline='')
csv_writer = csv.writer(data_file)

count = 0
for data in news['results']:
    if count == 0:
        header = data.keys()
        csv_writer.writerow(header)
        count += 1
    csv_writer.writerow(data.values())

data_file.close()

# **Using Newspaper3k to extract text from articles**

In [7]:
from newspaper import Article
import requests

def newspaper_text_extraction(article_url):
    # Set headers to simulate a browser request
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0'}

    # Check if the request is successful
    response = requests.get(article_url, headers=headers)

    if response.status_code == 200:  # If the response is successful, proceed with parsing
        article = Article(article_url)
        article.download()
        article.parse()
        return article.title, article.text
    else:
        print(f"Failed to retrieve {article_url} (status code: {response.status_code})")
        return None, None

news_contents = []

for i in range(0, 10):
    title, content = newspaper_text_extraction(news["results"][i]["link"])
    if content and len(content) > 100:
        print(news["results"][i]["link"])
        print(news["results"][i]["category"])
        news_contents.append({'title': title, 'content': content})


https://www.deccanherald.com/india/delhi/delhi-doctors-demand-justice-for-abhaya-safety-measures-for-healthcare-workers-3237339
['top']
https://www.outlookindia.com/sports/cricket/ind-vs-nz-1st-test-numbers-that-stand-out-from-indias-shocking-day-out-in-bengaluru-against-test-lowest-total-record-number-of-ducks
['top']
Failed to retrieve https://www.washingtontimes.com/news/2024/oct/17/renewed-push-for-assisted-dying-in-uk-faces-united/ (status code: 403)
https://www.freepressjournal.in/bhopal/centre-sanctions-3589-crore-for-bhopal-kanpur-economic-corridor
['top']
https://www.greaterkashmir.com/jammu/srs-congratulates-new-jk-govt/
['top']
https://timesofindia.indiatimes.com/city/noida/kolkata-man-arrested-for-running-fake-steel-company-website-to-scam-buyers-of-149-lakh/articleshow/114327389.cms
['top']
https://thesportsrush.com/tennis-news-casper-ruud-slammed-for-hypocrisy-after-comments-on-future-masters-1000-in-saudi-arabia/
['sports']
https://www.greaterkashmir.com/kashmir/mla-gulm

In [8]:
!pip install transformers
!pip install torch

from transformers import pipeline

In [9]:
!pip install pdfkit
!apt-get install -y wkhtmltopdf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wkhtmltopdf is already the newest version (0.12.6-2).
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.


# **Facebook Bart for summarization**

In [10]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
summarized_news = []

for n in news_contents:
  if isinstance(n["content"], str) and len(n["content"]) > 0:
    content = n["content"][:500]
    summary = summarizer(content, max_length=100, min_length=50, do_sample=False)
    print("--------------------------------------------------------------------------")
    print("Title : ", end="")
    print(n["title"])
    print("\n")
    print(summary[0]['summary_text'], end="\n\n")
    summarized_news.append({"title" : n["title"], "content" : summary[0]['summary_text'], "category" : ""})
  else:
    print("Invalid content at index:", n)

Your max_length is set to 100, but your input_length is only 50. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=25)


--------------------------------------------------------------------------
Title : Delhi doctors demand 'justice for Abhaya', safety measures for healthcare workers


Delhi doctors demand 'justice for Abhaya', safety measures for healthcare workers. Doctors held a protest outside the Banga Bhawan, holding posters and banners. They demanded swift justice for the murdered trainee doctor. The protest was called to demand swift justice and safety measures.

--------------------------------------------------------------------------
Title : IND Vs NZ 1st Test: Numbers That Stand Out From India’s Shocking Day Out In Bengaluru


Team India players stand for their national anthem before the start of the day two of the first cricket test match between India and New Zealand at the M.Chinnaswamy Stadium, in Bengaluru, India, Thursday, Oct. 17, 2024.

--------------------------------------------------------------------------
Title : Centre Sanctions ₹3,589 Crore For Bhopal-Kanpur Economic Corridor


# **Classifying the Summarized News**

In [12]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

nltk.download('punkt')
nltk.download('stopwords')

# Define the function to clean the news title column
def cleaned_desc_column(text):
  # Remove commas
  text = re.sub(r',', '', text)
  # Remove extra spaces
  text = re.sub(r'\s+', ' ', text)
  # Remove full stops
  text = re.sub(r'\.', '', text)
  # Remove single quotes and double quotes
  text = re.sub(r"['\"]", '', text)
  # Remove other non-word characters
  text = re.sub(r'\W', ' ', text)

  text_token = word_tokenize(text)
  stop_words = set(stopwords.words('english'))

  filtered_text = []

  for sw in text_token:
    if sw not in stop_words:
        filtered_text.append(sw)

  text = " ".join(filtered_text)
  return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### ***Loading news dataset for training classifier model***

In [13]:
import json

# List to store valid records
valid_records = []

# Open the JSON file and read line by line
with open('sample_data/newsData.json', 'r') as f:
    for line in f:
        try:
            # Try to parse each line as JSON
            record = json.loads(line)
            valid_records.append(record)
        except json.JSONDecodeError as e:
            # Skip lines with JSON errors and print a message
            print(f"Skipping malformed line: {line.strip()} - Error: {e}")

# Convert valid records into a DataFrame
news_data = pd.DataFrame(valid_records)

# Select the required columns if they exist
if 'headline' in news_data.columns and 'category' in news_data.columns:
    news_data = news_data[['headline', 'category']]

# Show the first few valid rows
print(news_data.head())

                                            headline   category
0  Over 4 Million Americans Roll Up Sleeves For O...  U.S. NEWS
1  American Airlines Flyer Charged, Banned For Li...  U.S. NEWS
2  23 Of The Funniest Tweets About Cats And Dogs ...     COMEDY
3  The Funniest Tweets From Parents This Week (Se...  PARENTING
4  Woman Who Called Cops On Black Bird-Watcher Lo...  U.S. NEWS


In [14]:
print(len(news_data))

209527


### ***Extracting only desired categories***

In [15]:
#unbalanced data we have down other category
min_samples=500
df_comedy=news_data[news_data.category=='COMEDY']
df_SPORTS =news_data[news_data.category=='SPORTS']
df_CRIME  =news_data[news_data.category=='CRIME']
df_ENTER =news_data[news_data.category=='ENTERTAINMENT']
df_EDU =news_data[news_data.category=='EDUCATION']
df_POLITICS =news_data[news_data.category=='POLITICS']
df_BUSSINESS =news_data[news_data.category=='BUSINESS'].sample(350,random_state=2020)
df_MEDIA =news_data[news_data.category=='MEDIA'].sample(220,random_state=2020)
df_GREEN =news_data[news_data.category=='GREEN'].sample(370,random_state=2020)
df_TECH =news_data[news_data.category=='TECH'].sample(140,random_state=2020)
df_ENVIRONMENT =news_data[news_data.category=='ENVIRONMENT'].sample(120,random_state=2020)

df_balanced=pd.concat([df_comedy,df_SPORTS,df_CRIME,df_ENTER,df_POLITICS, df_BUSSINESS, df_MEDIA, df_GREEN, df_TECH, df_ENVIRONMENT, df_EDU],axis=0)
df_balanced.category.value_counts()
news_data = df_balanced

In [16]:
news_data.category.value_counts()

,count
category,
POLITICS,35602
ENTERTAINMENT,17362
COMEDY,5400
SPORTS,5077
CRIME,3562
EDUCATION,1014
GREEN,370
BUSINESS,350
MEDIA,220


### ***Cleaning the news feed***

In [17]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
# Apply the clean_text_column function to the text_column in the DataFrame
news_data['headline'] = news_data['headline'].apply(cleaned_desc_column)
news_data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,headline,category
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY
344,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY
466,20 Of The Funniest Tweets About Cats And Dogs ...,COMEDY
510,Seth Meyers Has A Field Day With Rudy Giuliani...,COMEDY
536,25 Of The Funniest Tweets About Cats And Dogs ...,COMEDY


### ***Splitting the data into Training and Testing Dataset (80-20 split)***

In [18]:
#getting the category we need for testing
X = news_data['headline']
y = news_data['category']

import sklearn
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size = 0.20, random_state = 90)
print(X_train.shape)
print(X_test.shape)

(55373,)
(13844,)


### ***Using Logistic Regression for Prediction***

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline
lr = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LogisticRegression(max_iter = 1000)),])

# Train the logistic regression model on the training set
lr.fit(X_train,y_train)

# Make predictions on the test set
y_pred = lr.predict(X_test)

### ***Calculating the accuracy***

In [20]:
from sklearn.metrics import accuracy_score
print(f"Accuracy is: {accuracy_score(y_pred,y_test)}")

Accuracy is: 0.8275787344698065


In [21]:
summarized_news
for n in summarized_news:
  n["category"] = lr.predict([n["content"]])[0]

summarized_news[0]

{'title': "Delhi doctors demand 'justice for Abhaya', safety measures for healthcare workers",
 'content': "Delhi doctors demand 'justice for Abhaya', safety measures for healthcare workers. Doctors held a protest outside the Banga Bhawan, holding posters and banners. They demanded swift justice for the murdered trainee doctor. The protest was called to demand swift justice and safety measures.",
 'category': 'POLITICS'}

# **Creating PDFs**

In [22]:
!pip install reportlab

### ***PDF of original news***

In [23]:
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.units import inch



# Create a PDF document
pdf_output_path = "sample_data/original_news.pdf"
pdf = SimpleDocTemplate(pdf_output_path, pagesize=A4)

# Get default style for text
styles = getSampleStyleSheet()
content = []

# Add news titles and contents in sequence (without category)
for news in news_contents:
    # Add the news title
    content.append(Paragraph(f"<b>Title:</b> {news['title']}", styles['Heading2']))

    # Add the news content
    content.append(Paragraph(f"<b>Content:</b> {news['content']}", styles['BodyText']))

    # Add space between articles
    content.append(Spacer(1, 0.2 * inch))

# Build the PDF
pdf.build(content)

pdf_output_path

'sample_data/original_news.pdf'

### ***PDF of summarized news***

In [24]:
# Create a PDF document
pdf_output_path = "sample_data/summarized_news.pdf"
pdf = SimpleDocTemplate(pdf_output_path, pagesize=A4)

# Get default style for text
styles = getSampleStyleSheet()
content = []

# Group news by category
categories = {}
for news in summarized_news:
    category = news['category']
    if category not in categories:
        categories[category] = []
    categories[category].append(news)

# Add content to the PDF
for category, news_list in categories.items():
    # Add category title
    content.append(Paragraph(f"<b>Category: {category}</b>", styles['Title']))
    content.append(Spacer(1, 0.2 * inch))

    # Add news titles and contents under the category
    for news in news_list:
        content.append(Paragraph(f"<b>Title:</b> {news['title']}", styles['Heading2']))
        content.append(Paragraph(f"<b>Content:</b> {news['content']}", styles['BodyText']))
        content.append(Spacer(1, 0.2 * inch))

# Build the PDF
pdf.build(content)

pdf_output_path

'sample_data/summarized_news.pdf'